<a href="https://colab.research.google.com/github/Roxoner44/CalidadSoftware/blob/main/Snippets/CNN_Cervix_Snippets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Cargar Datos CNN Cervix

El uso de este colab es el de definir funciones y snipets que vamos a poder usar desde otros colabs, para no tener que caragrlos siempre

#Importar librerias CNN

In [ ]:
from google.colab import drive
#Montamos el drive en el directorio /content/drive
drive.mount("/content/drive")

%pip install -q -U keras-tuner
import tensorflow as tf
from tensorflow import keras

#KERAS
from keras import Sequential
from keras import models
from keras import layers
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import AveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout, BatchNormalization
from keras import optimizers

#UTILS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathlib
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import seaborn as sns

print(tf.__version__)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
2.9.2


#Carga de datos CNN como DataFrame

Vamos a definir dos tipos de cargas de datos, por DataFrame y por Directory.

Esto para poder escoger si cargar los datos como "one-hot encoding" o con "label encoding"

##Funciones para la carga de datos

In [ ]:
def cargarDataframeBinario(data):
  """
  Función que carga el dataframe con dos clases (benigna y altogrado) en lugar de con las clases originales

  Parameters:
    data (posixPath): path desde donde vamos a cargar las imagenes

  Returns:
    pandasDataframe: dataframe que contiene dos columnas (path y category) con todos los paths de las imagenes y sus categorias en formato binario (label encoding)

  """
  #Guardamos todos los paths que existen en el directorio que queremos
  lst = list(data.glob("*/*.tiff"))
  #Cramos una nueva lista
  newLst = list() 
  #Para cada elemento de la lista de paths (Por cada imagen)
  for l in lst:
      #Si el directorio en el que se encuentra es "benigna" significa que la celula no es revisable
      if l.parent == data / "benigna":
        tipo="No Revisable"
      #El resto son revisables
      else:
        tipo="Revisable"
      #Añade en la lista el tipo y el path de la imagen
      newLst.append((str(l),tipo))
  #Devuelve el dataframe generado a partir de la lista
  return  pd.DataFrame(newLst, columns = ['path', 'category'])

def cargarDataFrameCuatroClases(data):
  """
  Función que carga el dataframe con las clases originales

  Parameters:
    data (posixPath): path desde donde vamos a cargar las imagenes

  Returns:
    pandasDataframe: dataframe que contiene dos columnas (path y category) con todos los paths de las imagenes y sus categorias en formato binario (label encoding)

  """
  #Guardamos todos los paths que existen en el directorio que queremos
  lst = list(data.glob("*/*.tiff"))
  #Cramos una nueva lista
  newLst = list() 
  #Para cada elemento de la lista de paths (Por cada imagen)
  for l in lst:
    #Guarda el tipo como el nombre del directorio en el que se encuentra
    tipo=l.parent.name
     #Añade en la lista el tipo y el path de la imagen
    newLst.append((str(l),tipo)) #Devuelve el dataframe generado a partir de la lista
  return  pd.DataFrame(newLst, columns = ['path', 'category'])

In [ ]:


def  cargarDatosDataframe(num_classes :int, path = "/content/drive/My Drive/cérvix - proyecto/Celulas",pathResults="/content/drive/MyDrive/ucam/Investigación/resultados", batch_size : int =32,img_height :(int)=223,img_width: (int)=244):
  """Funcion que carga datos desde un dataframe, en contrario a cargarlo desde directorio directamente

  Parameters:
  num_classes (int): El numero de clases con las cuales se va a generar el dataframe (2 o 4)
  path: directorio en el cual se encuentran las imagenes originales que se van a cargar
  pathResults: directorio en el cual se van a enviar las imagenes de resultado (Si es que las hubiera)
  batch_size (int) =32: tamaño del batch de imagenes que vamos a usar, por defecto 32
  img_height (int)=223: alto de las imagenes que queremos
  img_width (int)=244: ancho de las imagenes que queremos


  Returns:
  train_generator : generator de las imagenes de entrenamiento a partir de dataframe
  validation_generator : generator de las imagenes de validacion a partir de dataframe
  test_generator : generator de las imagenes de test a partir de dataframe
  
  """
  CELULASPATH = path
  RESULTADOSPATH =pathResults
  #Guardamos el path en el cual se encuentran las imagenes de entrenamiento
  data_train = pathlib.Path(CELULASPATH + "/entrenamiento")

  if(num_classes==4):
    df = cargarDataframeBinario(data_train)
  else:
    df = cargarDataFrameCuatroClases(data_train)

  print(df.size())
  print(df.groupby('category').size())

  #Reescalamos los datos
  train_datagen = ImageDataGenerator(
      rescale = 1./255,
      horizontal_flip = True,
      vertical_flip = True,
      validation_split = 0.1)

  train_generator = train_datagen.flow_from_dataframe(
      df,
      x_col = "path",
      y_col="category",
      target_size = (img_height, img_width),
      batch_size = batch_size,
      shuffle = True,
      class_mode = 'sparse', #vamos a usar probar con binary
      subset = 'training',
      seed = 42)

  validation_generator = train_datagen.flow_from_dataframe(
      df,
      x_col = "path",
      y_col="category",
      target_size=(img_height, img_width),
      batch_size=batch_size,
      shuffle=True,
      class_mode='sparse',
      subset='validation',
      seed=42) # set as validation data

  #Guardamos el path en el cual se encuentran las imagenes de test
  data_test= pathlib.Path(CELULASPATH + "/test")

  #Reescalamos los datos
  test_datagen = ImageDataGenerator(rescale=1./255)

  test_generator = test_datagen.flow_from_dataframe(df_test,x_col = "path",y_col="category",
                                                target_size=(224, 224),
                                                batch_size=1,
                                                shuffle=True,
                                             class_mode='sparse')

  return train_generator,validation_generator,test_generator

#CargarDatos directory

##Funciones para cargar los datos

In [ ]:
def  cargarDatosDirectory(path = "/content/drive/My Drive/cérvix - proyecto/Celulas",pathResults="/content/drive/MyDrive/ucam/Investigación/resultados", batch_size : int =32,img_height :(int)=223,img_width: (int)=244):
  """Funcion que carga datos desde un directorio, te ccarga todas las imagenes que encuentre y guarda el label como el nombre del directorio

  Parameters:
  path: directorio en el cual se encuentran las imagenes originales que se van a cargar
  pathResults: directorio en el cual se van a enviar las imagenes de resultado (Si es que las hubiera)
  batch_size (int) =32: tamaño del batch de imagenes que vamos a usar, por defecto 32
  img_height (int)=223: alto de las imagenes que queremos
  img_width (int)=244: ancho de las imagenes que queremos


  Returns:
  train_generator : generator de las imagenes de entrenamiento a partir de directorio
  validation_generator : generator de las imagenes de validacion a partir de directorio
  test_generator : generator de las imagenes de test a partir de directorio
  
  """
  
  CELULASPATH = path
  RESULTADOSPATH =pathResults

  #Guardamos el path en el cual se encuentran las imagenes de entrenamiento
  data_train = pathlib.Path(CELULASPATH + "/entrenamiento")

  
 

  #Reescalamos los datos
  train_datagen = ImageDataGenerator(
      rescale = 1./255,
      horizontal_flip = True,
      vertical_flip = True,
      validation_split = 0.1)

  train_generator = train_datagen.flow_from_directory(
      data_train,
      target_size = (img_height, img_width),
      batch_size = batch_size,
      shuffle = True,
      class_mode = 'categorical',
      subset = 'training',
      seed = 42)

  validation_generator = train_datagen.flow_from_directory(
      data_train, # same directory as training data
      target_size=(img_height, img_width),
      batch_size=batch_size,
      shuffle=True,
      class_mode='categorical',
      subset='validation',
      seed=42) # set as validation data

  #Guardamos el path en el cual se encuentran las imagenes de test
  data_test= pathlib.Path(CELULASPATH + "/test")

  #Reescalamos los datos
  test_datagen = ImageDataGenerator(rescale=1./255)

  test_generator = test_datagen.flow_from_directory(data_test,
                                                  target_size=(224, 224),
                                                  batch_size=1,
                                                  shuffle=False,
  class_mode='categorical')

  return train_generator,validation_generator,test_generator

#Definir funciones para mostrar imagenes CNN

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

def mostrarConLabel(image_arr):
    x,y = image_arr.next()
    for i in range(0,3):
      image = x[i]
      label = y[i]
      print (label)
      plt.imshow(image)
      plt.show()

def mostrarConLabelMejor(image_arr,class_names):
  x,y = image_arr.next()
  plt.figure(figsize=(20,20))
  for i in range(10):
      plt.subplot(5,5,i+1)
      plt.xticks([])
      plt.yticks([])
      plt.grid(False)
      plt.imshow(x[i])
      for j in range(4):
         if(y[i][j]==1): break
      plt.title(class_names[j])
      
  plt.show()

#Mostrar datos

In [ ]:
#Recibe el history generado
def mostrarGrafico(history):
  acc      = history.history['accuracy' ]
  val_acc  = history.history[ 'val_accuracy' ]
  loss     = history.history[    'loss' ]
  val_loss = history.history['val_loss' ]

  epochs    = range(1,len(acc)+1,1) # obtener número de epochs

  plt.plot  ( epochs,     acc, 'r--', label='Training acc'  )
  plt.plot  ( epochs, val_acc,  'b', label='Validation acc')
  plt.title ('Training and validation accuracy')
  plt.ylabel('acc')
  plt.xlabel('epochs')

  plt.legend()
  plt.figure()

  plt.plot  ( epochs,     loss, 'r--' , label="Training Loss")
  plt.plot  ( epochs, val_loss ,  'b', label="Validation loss")
  plt.title ('Training and validation loss'   )
  plt.ylabel('loss')
  plt.xlabel('epochs')

  plt.legend()
  plt.figure()
  plt.show()
def mostrarMatrizConf(test_predict,test_generator, class_names):
  
  predict = []

  for i in test_predict:
    predict.append(int(np.argmax(i)))

  predict = np.asarray(predict)

  accuracy = accuracy_score(test_generator.classes, np.asarray(predict))
  print(accuracy)

  cm = confusion_matrix(test_generator.classes, predict)

  cm_display = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = class_names,)

  cm_display.plot(cmap = 'YlOrRd' )
  plt.show()

  #plt.figure(figsize = (10,10))
  #sns.heatmap(cm, annot=True, cmap='YlGn', xticklabels=class_names, yticklabels=class_names)
  #plt.show()
def calcularHistorial(model,history,validation_generator,test_generator,validation_steps):

  val_acc_per_epoch = history.history['val_accuracy']
  best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch))+1
  print('Mejor epoca: %d' % (best_epoch,))

  eval_result = model.evaluate(validation_generator, steps =validation_steps, verbose =1)
  
  print("[test loss, test accuracy]:", eval_result)
  test_predict = model.predict(test_generator, steps = test_generator.n // 1, verbose =1)
  class_names = list(validation_generator.class_indices.keys())
  mostrarMatrizConf(test_predict,test_generator,class_names)
  mostrarGrafico(history)


In [ ]:
def createBasicModel(num_classes: (int)=4, img_height: (int) = 244,img_width: (int)=244):
  model = tf.keras.Sequential([
      layers.Conv2D(32, 3, padding="same", activation="relu",input_shape=(img_height,img_width,3)),
      layers.Conv2D(32, 3, activation="relu"),
      layers.MaxPooling2D(2, 2),
      layers.Dropout(0.25),
      layers.Conv2D(64, 3, padding='same', activation='relu'),
      layers.Conv2D(64, 3, activation='relu'),
      layers.MaxPooling2D(2, 2),
      layers.Dropout(0.25),
      layers.Flatten(),
      layers.Dense(512, activation='relu'),
      layers.Dropout(0.1),
      layers.Dense(4, activation ="sigmoid")
  ])

  learning_rate=0.0001

  model.compile(optimizer= keras.optimizers.RMSprop(lr=learning_rate, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
  #model.compile(optimizer=keras.optimizers.Adam(learning_rate), loss='categorical_crossentropy',metrics=['accuracy'])
  model.summary()
  return model

#Definimos las funciones para el modelo CNN

In [ ]:
def ejemplo_ejecución():
  """
  batch_size = 32
  img_height = 244
  image_width = 244
  train_generator, validation_generator,test_generator = cargarDatosDirectory()

  steps_per_epoch = train_generator.n // batch_size
  print("Train Steps: "+str(steps_per_epoch)+ " Train Data: "+ str(train_generator.n)+ " Batch: "+ str(batch_size))
  validation_steps = validation_generator.n // batch_size
  print("Validation Steps: "+str(validation_steps)+ " Validation Data: "+ str(validation_generator.n)+ " Batch: "+ str(batch_size))

  model = createBasicModel()
  history = model.fit(train_generator,epochs=20,validation_data=validation_generator,validation_steps=validation_steps, shuffle=False)
  calcularHistorial(model,history,validation_generator,test_generator)
  """

  batch_size = 32
  img_height = 244
  img_width = 244
  train_generator, validation_generator,test_generator = cargarDatosDirectory()

  steps_per_epoch = train_generator.n // batch_size
  print("Train Steps: "+str(steps_per_epoch)+ " Train Data: "+ str(train_generator.n)+ " Batch: "+ str(batch_size))
  validation_steps = validation_generator.n // batch_size
  print("Validation Steps: "+str(validation_steps)+ " Validation Data: "+ str(validation_generator.n)+ " Batch: "+ str(batch_size))

  model = createBasicModel(img_height,img_width)
  history = model.fit(train_generator,epochs=20,validation_data=validation_generator,validation_steps=validation_steps, shuffle=False)
  calcularHistorial(model,history,validation_generator,test_generator)
